In [ ]:
#  相關套件

import time
import random
import json
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
from collections import deque
from concurrent.futures import ThreadPoolExecutor

WEB_NAME = '104 人力銀行 '
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36',
    'Referer': 'https://www.104.com.tw/jobs/search',
    }

In [ ]:
# 職業類別目錄

_json_url = "https://static.104.com.tw/category-tool/json/JobCat.json"

response = requests.get (_json_url)
json_data = response.json ()
df = pd.json_normalize (json_data)
df

# 資訊軟體系統類
# pd.json_normalize (df ['n'][6])

,no,des,n,eng
0,2001000000,經營／人資類,"[{'no': '2001001000', 'des': '經營／幕僚類人員', 'n': ...",Business/HR
1,2002000000,行政／總務／法務類,"[{'no': '2002001000', 'des': '行政／總務類人員', 'n': ...",Administrative/General Affairs/Legal Type
2,2003000000,財會／金融專業類,"[{'no': '2003002000', 'des': '金融專業相關類人員', 'n':...",Finance/Financial Professional
3,2004000000,行銷／企劃／專案管理類,"[{'no': '2004001000', 'des': '行銷類人員', 'n': [{'...",Marketing / Planning / Project Manager
4,2005000000,客服／門市／業務／貿易類,"[{'no': '2005001000', 'des': '客戶服務類人員', 'n': [...",Service / Store Clerk / Salesperson / Trade
5,2006000000,餐飲／旅遊 ／美容美髮類,"[{'no': '2006001000', 'des': '餐飲類人員', 'n': [{'...",Catering/Tourism/Beauty & Haircare Industry
6,2007000000,資訊軟體系統類,"[{'no': '2007001000', 'des': '軟體／工程類人員', 'n': ...",Information Software Systems
7,2010000000,操作／技術／維修類,"[{'no': '2010001000', 'des': '操作／技術類人員', 'n': ...",Operation / Technical / Maintenance Staff
8,2011000000,資材／物流／運輸類,"[{'no': '2011001000', 'des': '採購／資材／倉管類人員', 'n...",Materials/Logistics/Transportation
9,2012000000,營建／製圖類,"[{'no': '2012001000', 'des': '營建規劃類人員', 'n': [...",Construction/Drafting


In [ ]:
# 產生 104 人力銀行網址 https://www.104.com.tw 根據提供的 (關鍵字和職缺類別) 轉換為職缺網址

def catch_104_url (KEYWORDS, CATEGORY, ORDER=None):
    """
    這個函數會根據給定的關鍵字和類別參數構建一個完整的職缺網址。
    如果同時提供了關鍵字和類別，將會包含兩者；如果只提供其中一個，則只會包含該參數。

    參數:
    KEYWORDS (str): 職缺的關鍵字。
    CATEGORY (str): 職缺的類別。
    ORDER (int, optional): 排序的參數，預設為 None。

    返回:
    str: 生成的職缺網址。
    """

    BASE_URL = "https://www.104.com.tw/jobs/search/?jobsource=joblist_search&mode=s"
    params = ""  # 初始化 params 變數

    # 根據 ORDER 是否為 None 來決定是否添加 ORDER 參數
    if ORDER is not None:
        params += f"&order={ORDER}"
    if KEYWORDS:
        params += f"&keyword={KEYWORDS}"
    if CATEGORY:
        params += f"&jobcat={CATEGORY}"

    return f"{BASE_URL}{params}&page="


KEYWORDS_STR = "雲端工程師"
JOBCAT_CODE = "2007000000"
ORDER_SETTING = 15     # 15 (符合度高)、  16 (最近更新)


# 測試範例
# url_1 = catch_104_url (KEYWORDS_STR, JOBCAT_CODE, ORDER_SETTING)
# print (url_1)  # https://www.104.com.tw/jobs/search/?jobsource=joblist_search&mode=s&order=15&keyword=雲端工程師&jobcat=2007000000&page=

# url_2 = catch_104_url (KEYWORDS_STR, "")
# print (url_2)  # https://www.104.com.tw/jobs/search/?jobsource=joblist_search&mode=s&keyword=雲端工程師&page=

# url_3 = catch_104_url ("", JOBCAT_CODE, ORDER_SETTING)
# print (url_3)  # https://www.104.com.tw/jobs/search/?jobsource=joblist_search&mode=s&order=15&jobcat=2007000000&page=

# url_4 = catch_104_url ("","")
# print (url_4)  # https://www.104.com.tw/jobs/search/?jobsource=joblist_search&mode=s&page=

In [ ]:
#  從 api 網址獲取工作職缺的網址

def fetch_jobs_url (_CODE, KEYWORD):
    """
    這個函數會遍歷多個頁面，並從每個頁面中提取工作職缺的網址，將其存儲在一個集合中以避免重複。
    使用 tqdm 顯示進度條，並在每次請求之間隨機延遲以避免過於頻繁的請求。

    參數:
    _CODE (str): 職缺類別的代碼。
    KEYWORD (str): 搜尋的關鍵字。

    返回:
    list: 包含所有獲取到的工作職缺網址的列表。

    """

    BASE_URL = "https://www.104.com.tw/jobs/search/api/jobs"

    PAGE = 1
    MAX_PAGE = 10
    PAGE_SIZE = 30
    ORDER_SETTING = 15   # 15 (符合度高)、  16 (最近更新)
    
    MAX_LENGTH = 4
    recent_counts = deque (maxlen=MAX_LENGTH)
    job_url_set = set ()  # 用於存儲唯一的職缺網址

    with requests.Session () as session, tqdm (total=MAX_PAGE, desc="104 職缺列表", unit="PAGE", leave=True) as pbar:
        while True:
            params = {
                'jobsource': 'm_joblist_search',
                'page': PAGE,
                'pagesize': PAGE_SIZE,
                'order': ORDER_SETTING, 
                'jobcat': _CODE,
                'keyword': KEYWORD,
            }

            response = requests.get (BASE_URL, headers=HEADERS, params=params, timeout=20)
            api_job_urls = response.json ()['data']
            for job_url in api_job_urls:
                job_url_set.add (job_url ['link']['job'])

            # 檢查是否有新資料
            total_jobs = len (job_url_set) 
            recent_counts.append (total_jobs)
            if len (recent_counts) == MAX_LENGTH and len (set (recent_counts)) == 1:
                print (f"連續 {MAX_LENGTH} 次沒有新資料，提前結束。")
                break
            
            time.sleep (random.uniform (0.5, 1.5))
            
            pbar.set_postfix_str (f"目前頁面 {PAGE}, 最大頁數: {MAX_PAGE}")
            pbar.update (1)

            PAGE = PAGE + 1  # 更新頁碼
            if PAGE >= MAX_PAGE:
                MAX_PAGE = PAGE + 1 
                pbar.total = MAX_PAGE

    modified_job_url_set = {f"https://www.104.com.tw/job/ajax/content/{url.split ('/')[-1]}" for url in job_url_set}
    print (f"共獲取到 {len (job_url_set)} 筆職缺資料。")
    return list (modified_job_url_set)


# 測試範例
# JOBCAT_CODE = "2007000000"
# KEYWORDS = "雲端工程師"
# jobs_url = fetch_jobs_url (JOBCAT_CODE, KEYWORDS)
# jobs_url [0]

In [ ]:
# 從指定的職缺網址獲取職缺的相關數據

def fetch_job_data (job_url):
    """
    這個函數會發送 GET 請求到提供的職缺網址，並使用 BeautifulSoup 解析返回的 HTML 文檔。
    它會從頁面中的 JavaScript 代碼中提取職缺的元數據，並將其轉換為 Pandas DataFrame 格式。

    參數:
    job_url (str): 職缺的網址。

    返回:
    pd.DataFrame: 包含職缺詳細信息的 DataFrame，包括職缺網址、公司名稱、公司網址及其他職缺網址。
    """
    response = requests.get (job_url, headers=HEADERS)
    jobMetaData = response.json ()['data']
    df = pd.json_normalize (jobMetaData)
    return df

# 測試範例
# job_url = "https://www.104.com.tw/job/ajax/content/8k4lp"   # jobs_url [0]
# job_data = fetch_job_data (job_url)
# job_data

In [ ]:
# 根據關鍵字與職業類別 獲取所有工作職位的資料

SEARCH_TIMESTAMP = time.strftime ('%Y-%m-%d', time.localtime (time.time ()))
JOBCAT_CODE = "2007000000"
KEYWORDS = "雲端工程師"
FILE_NAME = f"({SEARCH_TIMESTAMP})_{WEB_NAME}_{KEYWORDS}_{JOBCAT_CODE}"

print ( f"開始執行 {FILE_NAME}" )
job_data_list = []
job_urls = fetch_jobs_url (JOBCAT_CODE, KEYWORDS)        # 列出 104 人力銀行 - 職缺網址列表 


all_jobs_df = pd.DataFrame ()  # 初始化一個空的 DataFrame

for url in tqdm (job_urls, desc="Fetching job data", unit="job"):
    df_job_data = fetch_job_data (url)
    all_jobs_df = pd.concat ([all_jobs_df, df_job_data], ignore_index=True)


all_jobs_df

開始執行 (2025-05-25)_104人力銀行_雲端工程師_2007000000


104職缺列表 :  95%|█████████▍| 37/39 [01:03<00:03,  1.72s/PAGE, 目前頁面 37, 最大頁數: 38]


連續4次沒有新資料，提前結束。
共獲取到 1024 筆職缺資料。


Fetching job data: 100%|██████████| 1024/1024 [04:03<00:00,  4.20job/s]


,switch,custLogo,postalCode,closeDate,industry,custNo,reportUrl,industryNo,employees,chinaCorp,...,jobDetail.startWorkingDay,jobDetail.hireType,jobDetail.delegatedRecruit,jobDetail.needEmp,jobDetail.landmark,jobDetail.remoteWork,interactionRecord.lastProcessedResumeAtTime,interactionRecord.nowTimestamp,jobDetail.remoteWork.type,jobDetail.remoteWork.description
0,on,https://static.104.com.tw/b_profile/cust_pictu...,220,,電腦軟體服務業,80004909000,https://www.104.com.tw/feedback?category=2&cus...,1001001002,350人,False,...,不限,0,,1人,距捷運新埔站約400公尺,None,1747995247,1748178629,NaN,NaN
1,on,https://static.104.com.tw/b_profile/cust_pictu...,115,,電腦軟體服務業,130000000091175,https://www.104.com.tw/feedback?category=2&cus...,1001001002,115人,False,...,不限,0,,2~3人,,None,1747883442,1748178629,2.0,每週四、五(通過試用期後)
2,on,https://static.104.com.tw/b_profile/cust_pictu...,106,2024-05-09,其他專業／科學及技術業,130000000207624,https://www.104.com.tw/feedback?category=2&cus...,1008003006,暫不提供,False,...,一個月內,0,,1人,,None,None,1748178630,NaN,NaN
3,on,https://static.104.com.tw/b_profile/cust_pictu...,804,,電腦軟體服務業,130000000221361,https://www.104.com.tw/feedback?category=2&cus...,1001001002,20人,False,...,不限,0,,1~2人,,None,None,1748178630,NaN,NaN
4,on,https://static.104.com.tw/b_profile/cust_pictu...,408,,電腦系統整合服務業,130000000226240,https://www.104.com.tw/feedback?category=2&cus...,1001001001,暫不提供,False,...,不限,0,,1~3人,距捷運文心森林公園站約330公尺,None,1747985120,1748178630,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019,on,https://static.104.com.tw/b_profile/cust_pictu...,242,,消費性電子產品製造業,23357403000,https://www.104.com.tw/feedback?category=2&cus...,1001003003,30000人,False,...,一個月內,0,,1人,距捷運新北產業園區站約350公尺,None,1747904654,1748178871,NaN,NaN
1020,on,https://static.104.com.tw/b_profile/cust_pictu...,221,,電腦及其週邊設備製造業,23718011000,https://www.104.com.tw/feedback?category=2&cus...,1001003001,770人,False,...,可年後上班,0,,1人,,None,None,1748178872,NaN,NaN
1021,on,https://static.104.com.tw/b_profile/cust_pictu...,106,,電腦軟體服務業,130000000130132,https://www.104.com.tw/feedback?category=2&cus...,1001001002,27人,False,...,不限,0,,1~4人,距捷運古亭站約420公尺,None,1748150944,1748178872,NaN,NaN
1022,on,https://static.104.com.tw/b_profile/cust_pictu...,104,,電腦系統整合服務業,70771557000,https://www.104.com.tw/feedback?category=2&cus...,1001001001,580人,False,...,一個月內,0,,1人,距捷運民權西路站約260公尺,None,None,1748178872,NaN,NaN


In [7]:
df.columns

Index(['switch', 'custLogo', 'postalCode', 'closeDate', 'industry', 'custNo',
       'reportUrl', 'industryNo', 'employees', 'chinaCorp',
       'corpImageRight.corpImageRight.imageUrl',
       'corpImageRight.corpImageRight.link', 'header.corpImageTop.imageUrl',
       'header.corpImageTop.link', 'header.jobName', 'header.appearDate',
       'header.custName', 'header.custUrl', 'header.analysisType',
       'header.analysisUrl', 'header.isSaved', 'header.isApplied',
       'header.applyDate', 'header.userApplyCount', 'header.isActivelyHiring',
       'contact.hrName', 'contact.email', 'contact.visit', 'contact.phone',
       'contact.other', 'contact.reply', 'environmentPic.environmentPic',
       'environmentPic.corpImageBottom.imageUrl',
       'environmentPic.corpImageBottom.link', 'condition.acceptRole.role',
       'condition.acceptRole.disRole.needHandicapCompendium',
       'condition.acceptRole.disRole.disability', 'condition.workExp',
       'condition.edu', 'condition.major'